<a href="https://www.kaggle.com/code/taciya/incfileobject?scriptVersionId=226251976" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import stat
import platform
import sys
from datetime import datetime

def print_file_info(file_path):
    try:
        # 检查文件是否存在
        if not os.path.exists(file_path):
            print(f"错误：文件 '{file_path}' 不存在")
            return

        # 获取文件基本状态信息
        file_stat = os.stat(file_path)
        
        # 文件类型判断
        file_type = "未知"
        if os.path.isfile(file_path):
            file_type = "普通文件"
        elif os.path.isdir(file_path):
            file_type = "目录"
        elif os.path.islink(file_path):
            file_type = "符号链接"

        # 文件系统编码检测
        fs_encoding = sys.getfilesystemencoding()

        # 获取时间信息（兼容不同系统）
        system = platform.system()
        #ctime = file_stat.st_ctime if system == 'Windows' else file_stat.st_birthtime
        ctime = file_stat.st_ctime 
        
        # 转换时间戳为可读格式
        time_format = "%Y-%m-%d %H:%M:%S"
        access_time = datetime.fromtimestamp(file_stat.st_atime).strftime(time_format)
        modify_time = datetime.fromtimestamp(file_stat.st_mtime).strftime(time_format)
        create_time = datetime.fromtimestamp(ctime).strftime(time_format) if ctime else "N/A"

        # 权限符号化处理
        permissions = stat.filemode(file_stat.st_mode)

        # 获取绝对路径和规范路径
        abs_path = os.path.abspath(file_path)
        real_path = os.path.realpath(file_path)

        # 构建信息字典
        file_info = {
            "文件名称": os.path.basename(file_path),
            "绝对路径": abs_path,
            "真实路径": real_path if real_path != abs_path else "N/A",
            "文件类型": file_type,
            "文件大小": f"{file_stat.st_size} bytes",
            "占用块数": file_stat.st_blocks,
            "IO块大小": f"{file_stat.st_blksize} bytes",
            "文件系统编码": fs_encoding,
            "访问权限": {
                "符号表示": permissions,
                "八进制表示": oct(file_stat.st_mode)[-4:]
            },
            "时间信息": {
                "创建时间": create_time,
                "修改时间": modify_time,
                "访问时间": access_time
            },
            "设备信息": {
                "inode编号": file_stat.st_ino,
                "设备号": f"{os.major(file_stat.st_dev)}:{os.minor(file_stat.st_dev)}",
                "硬链接数": file_stat.st_nlink
            },
            "所有者信息": {
                "用户ID": file_stat.st_uid,
                "组ID": file_stat.st_gid
            }
        }

        # 打印结构化信息
        print("\n文件详细信息：")
        print("=" * 50)
        for category, details in file_info.items():
            print(f"{category}:")
            if isinstance(details, dict):
                for k, v in details.items():
                    print(f"  ▸ {k.ljust(15)}: {v}")
            else:
                print(f"  ▸ {details}")
            print("-" * 50)

    except PermissionError:
        print(f"错误：没有访问 '{file_path}' 的权限")
    except Exception as e:
        print(f"发生未知错误: {str(e)}")